# Rejection-based EEG Signal Classification

## Import Libraries

In [20]:
import warnings

import matplotlib.pyplot as plt
import moabb
import numpy as np
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from moabb.datasets import BNCI2014_001, BNCI2014_004, Cho2017
from moabb.evaluations import (CrossSessionEvaluation, CrossSubjectEvaluation,
                               WithinSessionEvaluation)
from moabb.paradigms import LeftRightImagery, MotorImagery
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline

moabb.set_log_level("info")
warnings.filterwarnings("ignore")

## Instantiate a Dataset

In [21]:
dataset = BNCI2014_001()
dataset.subject_list = dataset.subject_list[:1]

datasets = [dataset]

An example of an EEG record obtained under the `mne` format, stored in a dictionary of sessions and runs.

In [22]:
sessions = dataset.get_data()

In [23]:
sessions

{1: {'0train': {'0': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '1': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '2': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '3': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '4': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '5': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>},
  '1test': {'0': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '1': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '2': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '3': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '4': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '5': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>}}}

## Choose a Paradigm

In [24]:
print(dataset.paradigm)

imagery


In [25]:
paradigm = MotorImagery(n_classes=4)

2024-04-04 18:29:39,519 WARNING MainThread moabb.paradigms.motor_imagery Choosing from all possible events


In [26]:
paradigm.datasets

## Unpack the Dataset

The data stored in the `sklearn` format.

In [27]:
X, y, meta = paradigm.get_data(dataset=dataset)

In [28]:
print(X.shape, type(X))

(576, 22, 1001) <class 'numpy.ndarray'>


In [29]:
print(y, "\n", type(y))
print("The number of labels:", len(y))

['tongue' 'feet' 'right_hand' 'left_hand' 'left_hand' 'right_hand' 'feet'
 'tongue' 'right_hand' 'feet' 'left_hand' 'left_hand' 'left_hand' 'tongue'
 'right_hand' 'right_hand' 'left_hand' 'left_hand' 'feet' 'left_hand'
 'right_hand' 'tongue' 'tongue' 'feet' 'left_hand' 'tongue' 'tongue'
 'right_hand' 'tongue' 'tongue' 'right_hand' 'left_hand' 'right_hand'
 'feet' 'feet' 'feet' 'tongue' 'feet' 'left_hand' 'tongue' 'right_hand'
 'feet' 'right_hand' 'feet' 'tongue' 'right_hand' 'feet' 'left_hand'
 'left_hand' 'left_hand' 'tongue' 'right_hand' 'left_hand' 'feet'
 'left_hand' 'feet' 'right_hand' 'tongue' 'left_hand' 'feet' 'feet'
 'left_hand' 'feet' 'right_hand' 'tongue' 'tongue' 'tongue' 'feet'
 'left_hand' 'tongue' 'right_hand' 'tongue' 'right_hand' 'left_hand'
 'feet' 'right_hand' 'left_hand' 'feet' 'feet' 'left_hand' 'feet' 'tongue'
 'tongue' 'right_hand' 'left_hand' 'right_hand' 'tongue' 'right_hand'
 'tongue' 'feet' 'right_hand' 'right_hand' 'right_hand' 'feet' 'tongue'
 'left_hand' '

In [30]:
print(meta)

     subject session run
0          1  0train   0
1          1  0train   0
2          1  0train   0
3          1  0train   0
4          1  0train   0
..       ...     ...  ..
571        1   1test   5
572        1   1test   5
573        1   1test   5
574        1   1test   5
575        1   1test   5

[576 rows x 3 columns]


In [31]:
print(np.unique(meta["session"]))

['0train' '1test']


## Create a Pipeline

In [32]:
pipelines = {}

pipelines["RG+LR"] = make_pipeline(Covariances(), TangentSpace(), LogisticRegression(solver="lbfgs"))

## Evaluate a Solution

In [33]:
# Evaluate for a specific number of training samples per class.
data_size = dict(policy="per_class", value=np.array([5, 10, 30, 50]))
# When the training data is sparse, perform more permutations than when we have a lot of data.
n_perms = np.floor(np.geomspace(20, 2, len(data_size["value"]))).astype(int)

In [34]:
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=datasets,
    overwrite=True,
    data_size=data_size,
    n_perms=n_perms,
)

In [35]:
results = evaluation.process(pipelines)

2024-04-04 18:29:42,940 INFO MainThread moabb.evaluations.base Processing dataset: BNCI2014-001
BNCI2014-001-WithinSession:   0%|          | 0/1 [00:00<?, ?it/s]2024-04-04 18:29:46,052 INFO MainThread moabb.evaluations.evaluations Permutation: 1, Training samples: 20
2024-04-04 18:29:46,096 INFO MainThread moabb.evaluations.base RG+LR | BNCI2014-001 | 1 | 0train: Score 0.707
2024-04-04 18:29:46,267 INFO MainThread moabb.evaluations.evaluations Permutation: 1, Training samples: 40
2024-04-04 18:29:46,330 INFO MainThread moabb.evaluations.base RG+LR | BNCI2014-001 | 1 | 0train: Score 0.690
2024-04-04 18:29:46,534 INFO MainThread moabb.evaluations.evaluations Permutation: 1, Training samples: 120
2024-04-04 18:29:46,680 INFO MainThread moabb.evaluations.base RG+LR | BNCI2014-001 | 1 | 0train: Score 0.776
2024-04-04 18:29:46,836 INFO MainThread moabb.evaluations.evaluations Permutation: 1, Training samples: 200
2024-04-04 18:29:47,036 INFO MainThread moabb.evaluations.base RG+LR | BNCI2014

In [36]:
results.head(n=5)

,score,time,samples,data_size,permutation,subject,session,channels,n_sessions,dataset,pipeline
0,0.706897,0.041033,20.0,20.0,1.0,1,0train,22,2,BNCI2014-001,RG+LR
1,0.689655,0.059046,40.0,40.0,1.0,1,0train,22,2,BNCI2014-001,RG+LR
2,0.775862,0.135094,120.0,120.0,1.0,1,0train,22,2,BNCI2014-001,RG+LR
3,0.775862,0.188067,200.0,200.0,1.0,1,0train,22,2,BNCI2014-001,RG+LR
4,0.741379,0.045160,20.0,20.0,2.0,1,0train,22,2,BNCI2014-001,RG+LR


In [37]:
results.to_csv("./results.csv")

In [38]:
results = pd.read_csv("./results.csv")

## Plot the Results

In [39]:
fig, ax = plt.subplots(facecolor="white", figsize=[8, 4])

n_subs = len(dataset.subject_list)

if n_subs > 1:
    r = results.groupby(["pipeline", "subject", "data_size"]).mean().reset_index()
else:
    r = results

sns.pointplot(data=r, x="data_size", y="score", hue="pipeline", ax=ax, palette="Set1")

errbar_meaning = "subjects" if n_subs > 1 else "permutations"
title_str = f"Error bar shows Mean-CI across {errbar_meaning}"
ax.set_xlabel("Amount of training samples")
ax.set_ylabel("ROC AUC")
ax.set_title(title_str)
fig.tight_layout()
plt.show()